# Data Preprocessing & Building The Model

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from statsmodels.tsa.seasonal import STL
from tensorflow.keras.layers import Dense,LSTM,Input,Concatenate,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [325]:
df=pd.read_csv(r'C:\Users\LENOVO\Desktop\Bakery sales.csv')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9148\795958931.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r'C:\Users\LENOVO\Desktop\Bakery sales.csv')


In [326]:
def onehot_encode(df, column):
    df = df.copy()
    df = df.groupby(['Date', 'Product', 'Price'])['Quantity'].sum()
    df = df.reset_index()
    dummies = pd.get_dummies(df[column], prefix='')
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    product_mapping = pd.DataFrame({'product': dummies.columns, 'product_index': range(len(dummies.columns))})
    product_mapping.to_csv(r'C:\Users\LENOVO\Desktop\product_mapping.csv', index=False)
    
    return df


In [327]:
df=onehot_encode(df,'Product')

In [328]:
def preprocess(df):
    df=df.copy()
    df['Date']=pd.to_datetime(df['Date'])
    df['year']=df['Date'].dt.year
    df['day']=df['Date'].dt.day
    df['month']=df['Date'].dt.month
    df.drop(columns=['Date'],inplace=True)
    return df

In [329]:
df=preprocess(df)

In [334]:
def apply_standard_scaler(df):
    cols_to_scale = [col for col in df.columns if col != "Quantity"]
    df=df[cols_to_scale].to_numpy()
    scaler = StandardScaler()
    df= scaler.fit_transform(df)
    with open(r"C:\Users\LENOVO\Desktop\Scaker.pkl", "wb") as f:
        pickle.dump(scaler, f)
    
    return df

In [335]:
scaled_df = apply_standard_scaler(df)
scaled_df

array([[-0.64612818, -0.01285564, -0.03728198, ..., -0.87646331,
        -0.67708838, -1.6630717 ],
       [-0.5074752 , -0.01285564, -0.03728198, ..., -0.87646331,
        -0.67708838, -1.6630717 ],
       [-0.59413331, -0.01285564, -0.03728198, ..., -0.87646331,
        -0.67708838, -1.6630717 ],
       ...,
       [ 0.25511616, -0.01285564, -0.03728198, ...,  1.14094908,
        -0.79079888,  0.97177235],
       [-0.5074752 , -0.01285564, -0.03728198, ...,  1.14094908,
        -0.79079888,  0.97177235],
       [-0.02218979, -0.01285564, -0.03728198, ...,  1.14094908,
        -0.79079888,  0.97177235]])

In [273]:
X=scaled_df.drop(columns=['Quantity'])
y=scaled_df['Quantity']

In [274]:
from sklearn.model_selection import train_test_split
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

In [275]:
model = Sequential()

In [276]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Input((153,)))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='mse', optimizer='adam', metrics=['mae'])
es = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
mc = ModelCheckpoint(r'C:\Users\LENOVO\Desktop\SalesForecastModel\best_model.h5', monitor='val_mae', mode='min', save_best_only=True)


In [277]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=100,callbacks=[es,mc])

Epoch 1/100
606/606 [==============================] - 12s 18ms/step - loss: 429.6600 - mae: 6.2660 - val_loss: 320.1612 - val_mae: 5.4706
Epoch 2/100
606/606 [==============================] - 10s 17ms/step - loss: 323.1827 - mae: 5.5030 - val_loss: 334.4818 - val_mae: 5.5063
Epoch 3/100
606/606 [==============================] - 10s 17ms/step - loss: 302.2151 - mae: 5.3207 - val_loss: 316.6090 - val_mae: 5.4767
Epoch 4/100
606/606 [==============================] - 10s 17ms/step - loss: 284.1058 - mae: 5.0852 - val_loss: 271.1682 - val_mae: 4.9176
Epoch 5/100
606/606 [==============================] - 10s 17ms/step - loss: 266.0034 - mae: 4.9376 - val_loss: 262.5542 - val_mae: 4.9855
Epoch 6/100
606/606 [==============================] - 11s 18ms/step - loss: 271.5858 - mae: 4.9851 - val_loss: 233.0727 - val_mae: 4.8071
Epoch 7/100
606/606 [==============================] - 11s 17ms/step - loss: 228.3220 - mae: 4.7113 - val_loss: 223.1884 - val_mae: 4.4937
Epoch 8/100
606/606 [======

# Using The Model

In [3]:
from tensorflow.keras.models import load_model
import pickle

product_mapping = pd.read_csv(r'C:\Users\LENOVO\Desktop\product_mapping.csv')
model = load_model(r'C:\Users\LENOVO\Desktop\SalesForecastModel\best_model.h5')

def predict(price, product_name, year, day, month):
    product_name = "_" + product_name
    product_index = product_mapping.loc[product_mapping['product'] == product_name, 'product_index'].iloc[0]
    product_vector = np.zeros(len(product_mapping))
    product_vector[product_index] = 1
    features = [price] + product_vector.tolist() + [year, day, month]
    features = np.array(features).reshape(1, -1)
    with open(r"C:\Users\LENOVO\Desktop\Scaker.pkl",'rb') as f:
        scaler = pickle.load(f)
    features_scaled = scaler.transform(features)
    y = model.predict(features_scaled)
    print(y)


In [24]:
# product_name=input("Enter Product Name: ")
# year=input("Enter Year of Prediction: ")
# month=input("Enter Month of Prediction: ")
# day=input("Enter Day of Prediction: ")
# price=input("Enter the price of the unit: ")
predict(2.1,'Moisson',year=2022,month=3,day=8)

1/1 [==============================] - 0s 19ms/step
[[4.4670506]]


In [9]:
dataframe=pd.read_csv(r'C:\Users\LENOVO\Desktop\Bakery sales.csv')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10684\3729809089.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe=pd.read_csv(r'C:\Users\LENOVO\Desktop\Bakery sales.csv')


In [10]:
dataframe=dataframe.groupby(['Date','Product','Price'])['Quantity'].sum().reset_index()

In [22]:
dataframe.sort_values(by='Quantity',ascending=True)[:20000]

,Date,Product,Price,Quantity
19720,6/4/2022,Financier,0.7,0.0
12745,4/20/2022,Traiteur,14.0,0.0
15464,5/2/2022,Baguette Apero,4.5,0.0
8893,3/11/2021,Triangles,2.5,0.0
18592,6/20/2022,Plat 7.60E,7.6,0.0
...,...,...,...,...
9323,3/16/2021,Croissant Amandes,1.4,6.0
9311,3/16/2021,Boule 200G,1.1,6.0
9309,3/16/2021,Banettine,0.6,6.0
11259,3/8/2022,Croissant Amandes,1.5,6.0
